In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
warnings.filterwarnings("ignore")

from bs4 import BeautifulSoup
import re
import time
import math

#드라이버 불러오기
path = 'chromedriver'
excutable_path = 'chromedriver.exe'
source_url = "https://map.kakao.com/"
driver = webdriver.Chrome(executable_path=excutable_path)

#인터넷 접속
driver.get(source_url)



In [10]:
import torch

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import numpy as np
import random
import time
import datetime


# 사용자 입력 받기

In [6]:
#사용자 입력 받기
place=input("리뷰를 보고 싶은 가게의 이름이 무엇인가요?")

#서치
search=driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]')
search.send_keys(place)
search.send_keys(Keys.ENTER)

driver.implicitly_wait(time_to_wait=5)

#카페 이름, url 뜯어오기

urls=[]
names=[]


def name_url():
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    moreviews = soup.find_all(name="a", attrs={"class":"moreview"}) #상세보기 모두 가져오기. 
    namelist=soup.find_all(name='a',attrs={"class":"link_name"}) #식당 이름 가져오기
    #근데 스크롤 안 내린 거 상에서만 다 가져온 것. 
    # < 눌러서 더 가져와야 함. 
    
    for i in moreviews:
        page_url=i.get("href")
        urls.append(page_url)

    for i in namelist:
        name=i.get("title")
        names.append(name)

name_url() #첫 페이지에서 이름 뜯어옴. 


리뷰를 보고 싶은 가게의 이름이 무엇인가요?블루보틀 성수


In [7]:

#첫 페이지의 첫 검색 결과를 바탕으로, 아래 코드를 실행
cafe=names[0]
url=urls[0]

total_review=pd.DataFrame(columns=["name","review","star","review_num"]) #전체 리뷰 저장할 데이터 프레임. 

driver.get(url) #링크 접속

t1=driver.page_source
t2=BeautifulSoup(t1, "html.parser")
t3=t2.find(name="div", attrs={"class":"evaluation_review"})
print("접속 완료")

driver.implicitly_wait(time_to_wait=5)

#후기가 없다면 그대로 나가기.
try:
    review_count=driver.find_element(By.CSS_SELECTOR,"#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div > a:nth-child(3) > span.color_g")
    if review_count.text=="(0)":
        print("후기 없음")
except:#후기 미제공
    print("후기 미제공")

#리뷰를 끝까지 다 내리기. 
#중복되는 후기가 너무 많음. =>근데 또 손으로 하나하나 눌러보면 괜찮음. =>implicitly_wait함

try:
    start=time.time()
    current=start
    more_info_button=driver.find_element(By.CSS_SELECTOR,'#mArticle > div.cont_evaluation > div.evaluation_review > a') #더보기 버튼
    while (current-start)<=5: #최대 5초까지만 리뷰 내리기. 
        if more_info_button.get_attribute("class")=='link_more link_unfold':
            break
        driver.execute_script("arguments[0].click();", more_info_button) 
        current=time.time()
    print("리뷰 내리기 완료")
    #오류: 못 나옴. 해결->css로 버튼에 접근. 
    if more_info_button.get_attribute("class")=='link_more':
        driver.execute_script("arguments[0].click();", more_info_button)
    driver.implicitly_wait(time_to_wait=2)
except:
    print("내릴 스크롤이 없음. ")
    pass

try:
    #리뷰 더보기 모두 클릭하기
    long_review_button=driver.find_elements(By.CSS_SELECTOR,'#mArticle > div.cont_evaluation > div.evaluation_review > ul > li > div.comment_info > p > button')

    for button in long_review_button:
        driver.execute_script("arguments[0].click();", button) 
        #개선 사항: 시간이 너무 오래 걸림. 버튼을 한번에 누를 방법이 없을까? 167개 누르는데 1분은 걸린듯. 
    print("리뷰 더보기 모두 클릭 완료")
except:
    print("누를 더보기 버튼 없음.")
    pass

#리뷰 모두 끌어오기 
t1=driver.page_source
t2=BeautifulSoup(t1, "html.parser")
t3=t2.find(name="div", attrs={"class":"evaluation_review"})

print("리뷰 모두 끌어오기 완료 ")


#별점 리스트 만들기
star=t3.find_all('span',{'class':'ico_star inner_star'}) #크롤링

star_list=[]
for i in star:
    real_star=int(i['style'].split(":")[1].split("%")[0])

    real_star/=20
    star_list.append(real_star)
#평균 별점 끌어오기<span class="txt_desc">3.6</span>/ 후기 끌어오기
ave_star=t3.find_all("span",{"class":"txt_desc"})
ave_star_list=[]
review_num_list=[]
index=0
for i in ave_star:
    if index%2!=0:
        ave_star_list.append(i.text)
    else:
        review_num_list.append(i.text)
    index+=1


#리뷰 크롤링
review=t3.find_all('p',{'class':'txt_comment'})
#리뷰 리스트 만들기 
review_list=[]
for i in review:
    real_review=i.text
    review_list.append(real_review)

df=pd.DataFrame(columns=["name","review","star"])
df["review"]=review_list
df["star"]=star_list
df["name"]=cafe
df["ave_star"]=ave_star_list
df["review_num"]=review_num_list
total_review=total_review.append(df,ignore_index=True)
print("한 가게 리뷰 크롤링 완료")



접속 완료
리뷰 내리기 완료
리뷰 더보기 모두 클릭 완료
리뷰 모두 끌어오기 완료 
한 가게 리뷰 크롤링 완료


# 데이터 전처리

In [8]:
df_1=total_review.drop_duplicates() #중복 제거
df_2=df_1[df_1["review"]!="더보기"] #리뷰 없는 데이터 제거
df_3=df_2.copy()

df_3["review"].replace('더보기$',"",inplace=True,regex=True) #더보기, 접기 없애기
df_3["review"].replace('접기$',"",inplace=True,regex=True)
df_4=df_3.copy()

df_4['review'] = df_4['review'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") #한글만 남기기.
df_5=df_4.copy() #df_4: 영어 리뷰 삭제

df_5["review"].replace('\s*$',"",inplace=True,regex=True) #공백으로만 이루어진 거 제거
df_5=df_5[~(df_5["review"]=="")] #리뷰 길이가 0인 거 제거.
df_5=df_5.drop_duplicates() #중복 제거


# 모델 불러오기

In [19]:

model=BertForSequenceClassification.from_pretrained("C:/Users/sycy0/숙탯_딥러닝_플젝/bert_initial.h5")

# 모델에 넣기 위한 전처리

In [22]:
tokenizer=BertTokenizer.from_pretrained("bert-base-multilingual-cased",do_lower_case=False) 
#다국어 버전 모델 내에 있는 토크나이저 사용/ 대소문자 구분 



In [26]:
def convert_data(raw):
    review_bert=["[CLS] "+str(s)+" [SEP]" for s in raw.review]
    tokenized_texts=[tokenizer.tokenize(s) for s in review_bert]

    MAX_LEN = 128
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

    attention_masks=[]
    for seq in input_ids:
        seq_mask=[float(i>0) for i in seq] #seq가 0이 아니면, 어텐션 마스크에 1을 저장한다.
        attention_masks.append(seq_mask)

    final_input=torch.tensor(input_ids)
    final_mask=torch.tensor(attention_masks)

    return final_input, final_mask

def test_review(review): 
    model.eval()
    inputs,masks=convert_data(review)
    b_input_ids=inputs
    b_input_mask=masks

    with torch.no_grad():
        outputs=model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask)

    #예측하기
    logits=outputs.logits
    return logits

# 긍정 확률 구하기

In [27]:
pd.set_option('display.max_colwidth', -1)


logits=test_review(df_5)

import torch.nn.functional as F
probabilities = F.softmax(logits, dim=-1)

prob_list=((probabilities[:,0]*100).tolist()) #반올림, 뒤에 %붙이기 필요

df_5["positive_prob"]=prob_list

df_5.sort_values(by="positive_prob",ascending=False,inplace=True)



# 결과 도출

In [28]:
print(place,"의 리뷰 중 긍정적인 리뷰는 다음과 같습니다.")
df_5.head(5)


블루보틀 성수 의 리뷰 중 긍정적인 리뷰는 다음과 같습니다.


,name,review,star,review_num,ave_star,positive_prob
139,블루보틀 성수 카페,주차 공간 너무 좋고 친절하고 쾌적하고 커피 맛도 좋았습니다 제 커피 내리면서 집중하고 직접 가져다 주시는데 이게 뭐라고 감동이죠,5.0,239,3.5,99.957832
63,블루보틀 성수 카페,핸드드립전문점 로스팅전문 드립커피 아메리카노 라떼 다 너무 좋음자리는 늘 부족,5.0,90,3.7,99.956444
5,블루보틀 성수 카페,놀라 아스크림 쫀쫀 고소 맛있어용,3.0,414,3.9,99.955444
153,블루보틀 성수 카페,직원분들 정말 친절하시고 핸드드립도 맛있음,5.0,16,3.8,99.955025
252,블루보틀 성수 카페,넘친절하게 설명도잘해주시고 세심함을 느꼈습니다,5.0,235,3.7,99.954910


In [29]:
print(place,"의 리뷰 중 부정적인 리뷰는 다음과 같습니다.")
df_5.tail(5)


블루보틀 성수 의 리뷰 중 부정적인 리뷰는 다음과 같습니다.


,name,review,star,review_num,ave_star,positive_prob
414,블루보틀 성수 카페,스벅 갈게,2.0,88,2.4,0.289019
94,블루보틀 성수 카페,집이 먼데 선물용이라 쇼핑백 구입한대도 안줌도쿄 블루보틀이 훨씬낫다육천원이 아까운 맛 맛없음,1.0,8,3.0,0.203658
193,블루보틀 성수 카페,드립이 먼저 제조됐는데 같이 주문한 아메리카노 순서가 아직 안됐다고 더 기다려 같이 받아야 된다 함 향 날아가고 식으며 드립 메리트 없어짐 아메리카노는 물양이 적다고 미리 양해 구하길래 뜨거운물 같이 달라니 안된다고 커피 다 마시고 나서 요청하면 줄 수 있다 함 위장에서 섞어야 함쟁반도 못 봄 컵만 들고 날라야 함그냥 모든 것이 회사입장에서 원칙이 적용되는 느낌직원들의 응대는 아주 친절하나,1.0,54,3.1,0.166801
231,블루보틀 성수 카페,커피맛없음 너무비쌈 자리불편 주차불가 커피빈 압승,1.0,14,3.3,0.119680
93,블루보틀 성수 카페,와이파이도 안되고 인테리어도 별로,2.0,119,3.7,0.117849
